# Note

- SensorTower unified_publisher and each ios/android app publisher are 2 different things. Because 1 game can have many apps for different markets and each app can have different publishers (Eg. Unified Publisher of Genshin is MiHoYo, VN Publisher of Genshin is Funtap, 2 totally different entities)

In [1]:
import pandas as pd
import requests
from pprint import pprint
from math import ceil
import copy
import random
from datetime import datetime, timedelta

# Supporting Functions

In [19]:
def parse_string_to_list(input_string):
    """
    Converts a string representation of a list into an actual Python list.

    Args:
        input_string (str): The input string representing a list, e.g., 
                            "['US', 'AU', 'CA']".

    Returns:
        list: A Python list parsed from the input string.
    """
    # Use eval to safely parse the string into a list
    try:
        result = eval(input_string)
        if isinstance(result, list):
            return result
        else:
            raise ValueError("Input string does not represent a valid list.")
    except Exception as e:
        raise ValueError(f"Failed to parse input string: {e}")

In [5]:
# (1) Get mapping publisher_name - publisher_id from local_app_info.xlsx

In [54]:
df_local_app_info = pd.read_excel(
    "data/output/local_app_info.xlsx",
    dtype=
    {'publisher_name': str,
     'publisher_id': str
    }
)

In [55]:
df_local_app_info

,app_id,unified_app_id,name,publisher_name,publisher_id,valid_countries,top_countries,publisher_country,release_date,country_release_date,os
0,370339723,55c502b302ac64f9c0001fdb,Texas Hold'em Poker: Pokerist,KamaGames,312479206,"['US', 'AU', 'CA', 'FR', 'DE', 'GB', 'IT', 'JP...","['US', 'RU', 'CN']",Ireland,2010-05-29T00:00:00Z,2010-05-29T00:00:00Z,ios
1,397553707,55c502b302ac64f9c0001fdb,Texas Poker: Pokerist Pro,KamaGames,312479206,"['US', 'AU', 'CA', 'FR', 'DE', 'GB', 'IT', 'JP...","['US', 'IT', 'FR']",Ireland,2010-12-08T00:00:00Z,2010-12-08T00:00:00Z,ios
2,403710610,55d93ce402ac645ad20bc510,Fishdom HD (Premium),Playrix,371962663,[],"['CN', 'US', 'DE']",Ireland,2010-11-25T00:00:00Z,2010-11-25T08:00:00Z,ios
3,406889139,56e35cf19429946c4e0bac66,Coin Master,Moon Active,470556954,"['US', 'AU', 'CA', 'FR', 'DE', 'GB', 'IT', 'JP...","['US', 'GB', 'VN']",Israel,2010-12-11T00:00:00Z,2010-12-11T00:00:00Z,ios
4,426911698,55d93ce402ac645ad20bc510,Fishdom HD,Playrix,371962663,[],"['US', 'RU', 'CN']",Ireland,2011-04-20T00:00:00Z,2011-04-19T11:59:50Z,ios
...,...,...,...,...,...,...,...,...,...,...,...
14806,com.ldwc.playwhale,622182bbd1de6214dc2d321a,Realm Guardian,Playwhale.Ltd,Playwhale.Ltd,"['US', 'AU', 'BR', 'CA', 'FR', 'DE', 'GB', 'JP...","['VN', 'BR', 'PH']",China,2022-02-28T00:00:00Z,2022-02-28T00:00:00Z,android
14807,com.mobirix.mbtkd,62304fb5c25b0006942eb5a6,Three Kingdoms Idle,mobirix,mobirix,"['US', 'AU', 'BR', 'CA', 'CN', 'FR', 'DE', 'GB...","['KR', 'VN', 'JP']",South Korea,2022-05-17T00:00:00Z,2022-05-17T00:00:00Z,android
14808,com.oimnim.DressupBeautyPerfect,621ad81508320d50741e56b2,Dressup Beauty Perfect Turn,CatriceBerkleyunoO109@gmail.com,CatriceBerkleyunoO109@gmail.com,"['CN', 'KR', 'DZ', 'AO', 'AT', 'AZ', 'BH', 'BB...",['TW'],NaN,2022-02-20T00:00:00Z,2022-02-20T00:00:00Z,android
14809,com.vtcmobile.tamlongquyet,61f3718ce9bbdb51a4e7e2ff,Tầm Long Quyết VTC,NPH VTC Mobile,NPH+VTC+Mobile,"['AU', 'BR', 'CA', 'CN', 'DE', 'GB', 'MX', 'RU...","['VN', 'TW', 'KR']",Vietnam,2022-02-22T00:00:00Z,2022-02-22T00:00:00Z,android


In [10]:
df_app_publisher_name_publisher_id = df_local_app_info[['publisher_id', 'publisher_name']].drop_duplicates()

In [11]:
df_app_publisher_name_publisher_id

,publisher_id,publisher_name
0,312479206,KamaGames
2,371962663,Playrix
3,470556954,Moon Active
5,431946155,Roblox Corporation
7,664666426,Nordeus
...,...,...
14803,Com2uS+Holdings+Corporation,Com2uS Holdings Corporation
14804,Travellet+VN,Travellet VN
14806,Playwhale.Ltd,Playwhale.Ltd
14808,CatriceBerkleyunoO109@gmail.com,CatriceBerkleyunoO109@gmail.com


In [2]:
# (2) Load table 'cleaned_publisher_name'

In [56]:
df_publisher_clean_data = pd.read_excel(
    "data/publisher/dim_unified_app_id_to_cleaned_publisher_name.xlsx", 
    sheet_name = "publisher_of_valid_vn_apps_per_",
    dtype=
    {'cleaned_publisher_name': str,
     'publishers': str
    }
)

In [57]:
df_publisher_clean_data

,unified_app_id,publishers,chosen_publisher,cleaned_publisher_name
0,53782e75830f782dbe000050,"['Funzio, Inc', 'Deca_Games']","Funzio, Inc","Funzio, Inc"
1,53782e84830f782dbe000053,"['PLAYTIKA UK - HOUSE OF FUN LIMITED', 'Playti...",PLAYTIKA UK - HOUSE OF FUN LIMITED,Playtika
2,53783014830f782dbe0000b8,"['DianDian Interactive Holding', 'Century Game...",Century Games PTE. LTD.,Century Games PTE. LTD.
3,5378307a830f782dbe0000d2,"['505 Go Inc.', '505 Go Inc.']",505 Go Inc.,505 Go Inc.
4,537831da830f782dbe000137,"['Glu Games LLC', 'Glu']",Glu Games LLC,Glu Games LLC
...,...,...,...,...
5010,665feab82ea732b1b61715fd,['Pham Tien Manh'],Pham Tien Manh,Pham Tien Manh
5011,6669c5864b5ce4f9f2239488,"['Funtap JSC', 'Funtap Games']",Funtap Games,Funtap
5012,666a7683850648f34fb41216,['Funtap JSC'],Funtap JSC,Funtap
5013,6671c114b2e31852277f76c2,"['HYPER HEAT', 'HYPER HEAT ENTERTAINMENT PTE. ...",HYPER HEAT,HYPER HEAT


In [58]:
# (3) Expand cleaned_publisher_name to all item in publishers (list) (through chosen_publisher table)

In [59]:
df_publisher_clean_data["original_publisher_name_list"] = df_publisher_clean_data["publishers"].apply(parse_string_to_list)

In [60]:
table_cleaned_publisher_name_publishers_list = df_publisher_clean_data[['cleaned_publisher_name', 'original_publisher_name_list']].to_dict(orient='records')

In [61]:
table_cleaned_publisher_name_publishers_list

[{'cleaned_publisher_name': 'Funzio, Inc',
  'original_publisher_name_list': ['Funzio, Inc', 'Deca_Games']},
 {'cleaned_publisher_name': 'Playtika',
  'original_publisher_name_list': ['PLAYTIKA UK - HOUSE OF FUN LIMITED',
   'Playtika UK – House of Fun Limited']},
 {'cleaned_publisher_name': 'Century Games PTE. LTD.',
  'original_publisher_name_list': ['DianDian Interactive Holding',
   'Century Games PTE. LTD.',
   'Century Games PTE. LTD.']},
 {'cleaned_publisher_name': '505 Go Inc.',
  'original_publisher_name_list': ['505 Go Inc.', '505 Go Inc.']},
 {'cleaned_publisher_name': 'Glu Games LLC',
  'original_publisher_name_list': ['Glu Games LLC', 'Glu']},
 {'cleaned_publisher_name': 'Episode Interactive',
  'original_publisher_name_list': ['Episode Interactive',
   'Episode Interactive']},
 {'cleaned_publisher_name': 'Gamebasics BV',
  'original_publisher_name_list': ['Gamebasics BV', 'Gamebasics BV']},
 {'cleaned_publisher_name': 'Kongregate',
  'original_publisher_name_list': ['Kong

In [62]:
table_cleaned_publisher_name_original_publisher_name = []

for row in table_cleaned_publisher_name_publishers_list:
    for original_name in row['original_publisher_name_list']:
        table_cleaned_publisher_name_original_publisher_name.append(
            {
                'cleaned_publisher_name': row['cleaned_publisher_name'],
                'original_publisher_name': original_name
            }
        )

In [63]:
table_cleaned_publisher_name_original_publisher_name

[{'cleaned_publisher_name': 'Funzio, Inc',
  'original_publisher_name': 'Funzio, Inc'},
 {'cleaned_publisher_name': 'Funzio, Inc',
  'original_publisher_name': 'Deca_Games'},
 {'cleaned_publisher_name': 'Playtika',
  'original_publisher_name': 'PLAYTIKA UK - HOUSE OF FUN LIMITED'},
 {'cleaned_publisher_name': 'Playtika',
  'original_publisher_name': 'Playtika UK – House of Fun Limited'},
 {'cleaned_publisher_name': 'Century Games PTE. LTD.',
  'original_publisher_name': 'DianDian Interactive Holding'},
 {'cleaned_publisher_name': 'Century Games PTE. LTD.',
  'original_publisher_name': 'Century Games PTE. LTD.'},
 {'cleaned_publisher_name': 'Century Games PTE. LTD.',
  'original_publisher_name': 'Century Games PTE. LTD.'},
 {'cleaned_publisher_name': '505 Go Inc.',
  'original_publisher_name': '505 Go Inc.'},
 {'cleaned_publisher_name': '505 Go Inc.',
  'original_publisher_name': '505 Go Inc.'},
 {'cleaned_publisher_name': 'Glu Games LLC',
  'original_publisher_name': 'Glu Games LLC'},


In [64]:
df_cleaned_publisher_name_original_publisher_name = pd.DataFrame(table_cleaned_publisher_name_original_publisher_name).drop_duplicates().reset_index(drop=True)

In [65]:
df_cleaned_publisher_name_original_publisher_name

,cleaned_publisher_name,original_publisher_name
0,"Funzio, Inc","Funzio, Inc"
1,"Funzio, Inc",Deca_Games
2,Playtika,PLAYTIKA UK - HOUSE OF FUN LIMITED
3,Playtika,Playtika UK – House of Fun Limited
4,Century Games PTE. LTD.,DianDian Interactive Holding
...,...,...
3697,"Guangzhou Kangwei Information Technology Co., ...","Guangzhou Kangwei Information Technology Co., ..."
3698,Pham Tien Manh,Pham Tien Manh
3699,HYPER HEAT,HYPER HEAT
3700,HYPER HEAT,HYPER HEAT ENTERTAINMENT PTE. LTD.


In [66]:
# (4) Link publisher_id to cleaned_publisher_name (1) & (3)

In [67]:
df_publisher_id_cleaned_publisher_name = pd.merge(
    df_app_publisher_name_publisher_id,
    df_cleaned_publisher_name_original_publisher_name,
    left_on = 'publisher_name',
    right_on = 'original_publisher_name',
    how = 'left'
)

In [70]:
df_publisher_id_cleaned_publisher_name

,publisher_id,publisher_name,cleaned_publisher_name,original_publisher_name
0,312479206,KamaGames,KamaGames,KamaGames
1,371962663,Playrix,VTC,Playrix
2,470556954,Moon Active,VTC,Moon Active
3,431946155,Roblox Corporation,NaN,NaN
4,664666426,Nordeus,NaN,NaN
...,...,...,...,...
5996,Com2uS+Holdings+Corporation,Com2uS Holdings Corporation,NaN,NaN
5997,Travellet+VN,Travellet VN,NaN,NaN
5998,Playwhale.Ltd,Playwhale.Ltd,Playwhale.Ltd,Playwhale.Ltd
5999,CatriceBerkleyunoO109@gmail.com,CatriceBerkleyunoO109@gmail.com,NaN,NaN


In [72]:
df_publisher_id_cleaned_publisher_name.to_excel('data/output/df_publisher_id_cleaned_publisher_name.xlsx', index=False)